In [97]:
import numpy as np
#import pandas as pd
from sklearn import model_selection, svm, datasets
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import matplotlib.figure as figure
import time

In [98]:
# Settings
svrcs = 2**np.arange( -5, 11, dtype=float)          # Candidates of C
svrepsilons = 2**np.arange( -10, 1, dtype=float)    # Candidates of epsilon
svrgammas = 2**np.arange( -20, 11, dtype=float)     # Candidates of gamma
foldnumber = 5 # "foldnumber"-fold cross-validation
nmberoftrainingsamples = 1000
nmberoftestsamples = 0.25

In [99]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

import pandas as pd
import math
import numpy as np
from sklearn import tree

In [100]:
team_name_all=["愛媛","横浜","岡山","岐阜","京都","金沢","熊本","甲府","山形","山口","讃岐","鹿児島","松本","湘南","新潟","水戸","千葉","群馬","大宮","大分","町田","長崎","東京","徳島","栃木","柏","福岡","名古屋","琉球"]
team_number_all=["37","34","42","39","24","275","38","28","29","330","48","338","46","12","78","94","2","35","27","31","45","47","4","36","40","11","23","8","277"]

In [101]:
all_team=[]
for i in range(29):
    team_name=team_name_all[i]
    team_number=team_number_all[i]
    url=f"https://data.j-league.or.jp/SFMS01/search?competition_years=2019&competition_years=2018&competition_years=2017&competition_frame_ids=2&team_ids={team_number}&home_away_select=1&tv_relay_station_name="
    fomerdata=pd.read_html(url,header=0)
    data=fomerdata[0].drop(["大会","スタジアム"],axis=1)
    game_day=data.試合日
    game_day=[s.replace("月・祝","月").replace("火・祝","火").replace("水・祝","水").replace("木・祝","木").replace("金・祝","金").replace("土・祝","土").replace("日・祝","日").replace("月・休","月") for s in game_day]
    day=data.年度.astype(str)+"/"+game_day
    day=pd.DataFrame({"年/月/日":day})
    data=pd.concat([data,day],axis=1)
    tenki=pd.read_excel(f"c:\\Users\\梅津魁秀\\Desktop\\卒業研究\\天気データ\\全国天気\\{team_name}.xlsx",skiprows=3)
    tenki=tenki.fillna(0)
    tenki=tenki.drop(tenki.index[[0]]).reset_index(drop=True)
    tenki=tenki.drop(tenki.index[1095])
    google_trend=pd.read_csv("gt.csv").drop(["Unnamed: 0"],axis=1)
    google_trend=google_trend[f"{team_name}"]
    tenki=pd.concat([tenki,google_trend],axis=1)
    tenki=tenki.rename(columns={f"{team_name}":"ホームトレンド"})
    day2=tenki.年.round().astype(int).astype(str)+"/"+tenki.月.round().astype(int).astype(str).str.zfill(2)+"/"+tenki.日.round().astype(int).astype(str).str.zfill(2)+"("+tenki.曜日+")"
    day2=pd.DataFrame({"年/月/日":day2})
    tenki=pd.concat([tenki,day2],axis=1)
    data=pd.merge(data,tenki[["年/月/日","平均気温(℃)","降水量の合計(mm)","日照時間(時間)","ホームトレンド"]],on="年/月/日",how="left")
    sec=data.節.str.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
    sec=sec.str.extract("(.+)節",expand=True).rename(columns={0:"節数"})
    data=pd.concat([data,sec],axis=1)
    rank=pd.read_csv("c:\\Users\\梅津魁秀\\anaconda3\\rank.csv").drop(["Unnamed: 0"],axis=1)
    rank2=rank.rename(columns={"アウェイ":"ホーム"})
    data=pd.merge(data,rank[["アウェイ","節数","年度","順位"]],on=["アウェイ","年度","節数"],how="left")
    data=pd.merge(data,rank2[["ホーム","節数","年度","順位"]],on=["ホーム","年度","節数"],how="left")
    data=data.rename(columns={"順位_x":"アウェイ順位","順位_y":"ホーム順位"})
    data["節数"]=data["節数"].str.replace('第', '')
    data["K/O時刻"]=data["K/O時刻"].str[:2]
    score=data.スコア.str.split('-', expand=True)
    score=score[[0,1]].astype(np.int64)
    data["点数(ホームーアウェイ)"]=score[0]-score[1]
    data=data.drop(["年度","節","試合日","ホーム","インターネット中継・TV放送","年/月/日","アウェイ","スコア"],axis=1)
    #入場者数と平均気温に関しては、その地域ごとに偏差をとり、データに反映する。
    visitors=int(data["入場者数"].mean())
    temperature＿mean=int(data["平均気温(℃)"].mean())
    #それぞれ平均を引いた値を出力する。
    data["平均気温(℃)"]=data["平均気温(℃)"]-temperature＿mean
    data["入場者数"]=data["入場者数"]-visitors
    all_team.append(data)

In [102]:
#forループでリストにしっかり格納されたか個数をチェック(29個であれば成功)
print(len(all_team))
#リストを全て合わせてall_team_kanseiとする。
all_team_kansei=pd.concat(all_team)
#データを行方向でシャッフル
#再度reset_indexを↓で行うことで、行番号を再度設定する。
all_team_kansei=all_team_kansei.sample(frac=1)
all_team_kansei=all_team_kansei.astype('float64').reset_index(drop=True)
all_team_kansei

29


,K/O時刻,入場者数,平均気温(℃),降水量の合計(mm),日照時間(時間),ホームトレンド,節数,アウェイ順位,ホーム順位,点数(ホームーアウェイ)
0,14.0,481.0,-8.6,0.0,12.6,48.0,9.0,2.0,10.0,0.0
1,14.0,702.0,3.8,0.5,6.5,48.0,33.0,22.0,4.0,1.0
2,17.0,-3481.0,-2.5,9.5,0.0,18.0,7.0,21.0,4.0,2.0
3,19.0,-2081.0,8.4,45.5,0.0,38.0,23.0,13.0,15.0,0.0
4,18.0,-1000.0,2.8,6.0,0.0,21.0,19.0,4.0,11.0,-3.0
...,...,...,...,...,...,...,...,...,...,...
1381,15.0,2284.0,1.4,0.0,6.7,34.0,35.0,5.0,11.0,-1.0
1382,15.0,481.0,2.7,0.0,7.8,50.0,17.0,14.0,22.0,-2.0
1383,19.0,-2220.0,1.9,7.5,7.7,13.0,18.0,19.0,5.0,1.0
1384,14.0,588.0,1.4,8.5,5.9,60.0,12.0,4.0,18.0,0.0


In [103]:
#説明変数となるものをそれぞれx1~x9まで作成
#平均気温、降水量のデータは外したほうがよさそう（だからと言って、結果が良くなるわけではないが）
x1=np.array(all_team_kansei["K/O時刻"])
x2=np.array(all_team_kansei["平均気温(℃)"])
x3=np.array(all_team_kansei["降水量の合計(mm)"])
x4=np.array(all_team_kansei["日照時間(時間)"])
x5=np.array(all_team_kansei["節数"])
x6=np.array(all_team_kansei["アウェイ順位"])
x7=np.array(all_team_kansei["ホーム順位"])
x8=np.array(all_team_kansei["点数(ホームーアウェイ)"])
x9=np.array(all_team_kansei["ホームトレンド"])
#目的変数(観客動員数)をyに入れる
y=np.array(all_team_kansei["入場者数"])
X=np.c_[x1,x2,x3,x4,x5,x6,x7,x8,x9]#SVRが使えるように変数を結合
print(X.dtype,y.dtype)

float64 float64


In [104]:
# Divide samples into trainign samples and test samples
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=nmberoftestsamples, random_state=0)

In [105]:
# Standarize X and y
autoscaledXtrain = (Xtrain - Xtrain.mean(axis=0)) / Xtrain.std(axis=0, ddof=1)
autoscaledytrain = (ytrain - ytrain.mean()) / ytrain.std(ddof=1)
autoscaledXtest =  (Xtest - Xtrain.mean(axis=0)) / Xtrain.std(axis=0, ddof=1)

In [106]:
# Measure time in hyperparameter optimization
starttime = time.time()

In [107]:
# Optimize gamma by maximizing variance in Gram matrix
numpyautoscaledXtrain = np.array(autoscaledXtrain)
varianceofgrammatrix = list()
for svrgamma in svrgammas:
    grammatrix = np.exp(-svrgamma*((numpyautoscaledXtrain[:, np.newaxis] - numpyautoscaledXtrain)**2).sum(axis=2))
    varianceofgrammatrix.append(grammatrix.var(ddof=1))
optimalsvrgamma = svrgammas[ np.where( varianceofgrammatrix == np.max(varianceofgrammatrix) )[0][0] ]


In [108]:
# Optimize epsilon with cross-validation
svrmodelincv = GridSearchCV(svm.SVR(kernel='rbf', C=3, gamma=optimalsvrgamma), {'epsilon':svrepsilons}, cv=foldnumber )
svrmodelincv.fit(autoscaledXtrain, autoscaledytrain)
optimalsvrepsilon = svrmodelincv.best_params_['epsilon']

In [109]:
# Optimize C with cross-validation
svrmodelincv = GridSearchCV(svm.SVR(kernel='rbf', epsilon=optimalsvrepsilon, gamma=optimalsvrgamma), {'C':svrcs}, cv=foldnumber )
svrmodelincv.fit(autoscaledXtrain, autoscaledytrain)
optimalsvrc = svrmodelincv.best_params_['C']

In [110]:
# Optimize gamma with cross-validation (optional)
svrmodelincv = GridSearchCV(svm.SVR(kernel='rbf', epsilon=optimalsvrepsilon, C=optimalsvrc), {'gamma':svrgammas}, cv=foldnumber )
svrmodelincv.fit(autoscaledXtrain, autoscaledytrain)
optimalsvrgamma = svrmodelincv.best_params_['gamma']

In [111]:
# Check time in hyperparameter optimization
elapsedtime = time.time() - starttime
print ("Elapsed time in hyperparameter optimization: {0} [sec]".format(elapsedtime))

Elapsed time in hyperparameter optimization: 75.80478167533875 [sec]


In [112]:
# Check optimized hyperparameters
print ("C: {0}, Epsion: {1}, Gamma: {2}".format(optimalsvrc, optimalsvrepsilon, optimalsvrgamma))


C: 1.0, Epsion: 0.5, Gamma: 0.0625


In [113]:
# Construct SVR model
regressionmodel = svm.SVR(kernel='rbf', C=optimalsvrc, epsilon=optimalsvrepsilon, gamma=optimalsvrgamma)
regressionmodel.fit(autoscaledXtrain, autoscaledytrain)

SVR(epsilon=0.5, gamma=0.0625)

In [114]:
# Calculate y of trainig dataset
calculatedytrain = np.ndarray.flatten( regressionmodel.predict(autoscaledXtrain) )
calculatedytrain = calculatedytrain*ytrain.std(ddof=1) + ytrain.mean()

In [115]:
# r2, RMSE, MAE
print( "r2: {0}".format(float( 1 - sum( (ytrain-calculatedytrain )**2 ) / sum((ytrain-ytrain.mean())**2) )) )
print( "RMSE: {0}".format(float( ( sum( (ytrain-calculatedytrain)**2 )/ len(ytrain))**0.5 )) )
print( "MAE: {0}".format(float( sum( abs(ytrain-calculatedytrain)) / len(ytrain) )) )

r2: 0.28912722001221713
RMSE: 1940.1211834844644
MAE: 1363.8655021974664


In [116]:
# Estimate y in cross-validation
estimatedyincv = np.ndarray.flatten( model_selection.cross_val_predict(regressionmodel, autoscaledXtrain, autoscaledytrain, cv=foldnumber) )
estimatedyincv = estimatedyincv*ytrain.std(ddof=1) + ytrain.mean()

In [117]:
# r2cv, RMSEcv, MAEcv
print( "r2cv: {0}".format(float( 1 - sum( (ytrain-estimatedyincv )**2 ) / sum((ytrain-ytrain.mean())**2) )) )
print( "RMSEcv: {0}".format(float( ( sum( (ytrain-estimatedyincv)**2 )/ len(ytrain))**0.5 )) )
print( "MAEcv: {0}".format(float( sum( abs(ytrain-estimatedyincv)) / len(ytrain) )) )

r2cv: 0.19777171452006703
RMSEcv: 2061.01854884221
MAEcv: 1460.7323695806401


In [118]:
# Estimate y of test dataset
predictedytest = np.ndarray.flatten( regressionmodel.predict(autoscaledXtest) )
predictedytest = predictedytest*ytrain.std(ddof=1) + ytrain.mean()

In [119]:
# r2p, RMSEp, MAEp
print( "r2p: {0}".format(float( 1 - sum( (ytest-predictedytest )**2 ) / sum((ytest-ytest.mean())**2) )) )
print( "RMSEp: {0}".format(float( ( sum( (ytest-predictedytest)**2 )/ len(ytest))**0.5 )) )
print( "MAEp: {0}".format(float( sum( abs(ytest-predictedytest)) / len(ytest) )) )

r2p: 0.16122109179558075
RMSEp: 2398.7946699632107
MAEp: 1576.8836700032148
